In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [15]:
train_data = pd.read_csv("../../Data/titanic/train.csv")
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [16]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


    注意：存在缺失值，并且Cabin这一列缺失值较多，但是其不作为特征值，不做处理
         我们选取的特征值：pclass、sex、age，
         这三个特征值存在缺失值也较多，我们样本的数据量又较大，
         不可以直接删除，要进行替换——平均值、中位数
         因为缺失值处不存在其他的特殊符号，所以就省去了缺失值转换这一步骤

In [17]:
# 2.1.确定目标值、特征值
x = train_data[["Pclass", "Sex", "Age"]]
y = train_data["Survived"]

In [20]:
# 2.2.缺失值处理
x["Age"].fillna(value=train_data["Age"].mean(), inplace=True)
x

C:\Users\张耀翔\AppData\Local\Temp\ipykernel_9156\1959112411.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x["Age"].fillna(value=train_data["Age"].mean(), inplace=True)
C:\Users\张耀翔\AppData\Local\Temp\ipykernel_9156\1959112411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["Age"].fillna(value=train_data["Age"].mean(), inplace=True)


,Pclass,Sex,Age
0,3,male,22.000000
1,1,female,38.000000
2,3,female,26.000000
3,1,female,35.000000
4,3,male,35.000000
...,...,...,...
886,2,male,27.000000
887,1,female,19.000000
888,3,female,29.699118
889,1,male,26.000000


In [21]:
# 2.3.数据集分割
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    train_size=0.75,
    random_state=22
)

In [22]:
# 3.特征工程——字典特征提取
# 3.1.类型转换
x_train = x_train.to_dict(orient="records")
x_test = x_test.to_dict(orient="records")

In [23]:
x_train

[{'Pclass': 2, 'Sex': 'female', 'Age': 28.0},
 {'Pclass': 1, 'Sex': 'male', 'Age': 45.5},
 {'Pclass': 1, 'Sex': 'male', 'Age': 38.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 29.69911764705882},
 {'Pclass': 3, 'Sex': 'female', 'Age': 18.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 30.0},
 {'Pclass': 3, 'Sex': 'female', 'Age': 32.0},
 {'Pclass': 3, 'Sex': 'female', 'Age': 45.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 4.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 31.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 29.69911764705882},
 {'Pclass': 3, 'Sex': 'male', 'Age': 22.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 27.0},
 {'Pclass': 3, 'Sex': 'female', 'Age': 29.69911764705882},
 {'Pclass': 3, 'Sex': 'male', 'Age': 43.0},
 {'Pclass': 1, 'Sex': 'male', 'Age': 37.0},
 {'Pclass': 3, 'Sex': 'male', 'Age': 18.0},
 {'Pclass': 3, 'Sex': 'female', 'Age': 14.0},
 {'Pclass': 3, 'Sex': 'female', 'Age': 10.0},
 {'Pclass': 2, 'Sex': 'female', 'Age': 27.0},
 {'Pclass': 1, 'Sex': 'female', 'Age': 41.0},
 {'Pclass': 3, 'Sex'

In [24]:
# 3.2.特征工程——字典特征提取
transfer = DictVectorizer(sparse=True)
x_train = transfer.fit_transform(x_train)
x_test = transfer.fit_transform(x_test)

In [25]:
x_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2004 stored elements and shape (668, 4)>

In [26]:
# 4.机器学习
estimator = DecisionTreeClassifier()
estimator.fit(x_train, y_train)

DecisionTreeClassifier()

In [27]:
# 5.模型评估
y_pre = estimator.predict(x_test)
y_pre

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1])

In [29]:
ret = estimator.score(x_test, y_test)
ret

0.7802690582959642

In [31]:
export_graphviz(estimator, out_file="Titanic_tree.dot", feature_names=["Age", "Pclass","女性","男性"])